In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark=SparkSession.builder.appName('EY-Prep1').getOrCreate()
spark

In [10]:
#Handle data with different delimeter
data=["1,Alice\t30|New York"]
#data=["vivek","Akash","Supriya"]
#data=[(1,"vivek"),(2,"supriya"),(3,"akash")]
#data=[("vivek"),("supriya"),("akash")]
#data=["1,Alice,30,New York"]
df=spark.createDataFrame(data,"string")
df.show()

+--------------------+
|               value|
+--------------------+
|1,Alice\t30|New York|
+--------------------+



In [11]:
df.withColumn("split_string",split(col("value"),",|\t|\|")).select(col("split_string").getItem(0).alias("id"),\
                                                             col("split_string").getItem(1).alias("Name"),\
                                                             col("split_string").getItem(2).alias("Age"),\
                                                             col("split_string").getItem(3).alias("City")).show()

+---+-----+---+--------+
| id| Name|Age|    City|
+---+-----+---+--------+
|  1|Alice| 30|New York|
+---+-----+---+--------+



In [28]:
df.withColumn("id",split(col("value"),",").getItem(0)).\
   withColumn("Name",split(col("value"),",").getItem(1)).\
   withColumn("Age",split(col("value"),",").getItem(2)).\
   withColumn("City",split(col("value"),",").getItem(3)).select("id","Name","Age","City").show()

+---+------------------+----+----+
| id|              Name| Age|City|
+---+------------------+----+----+
|  1|Alice\t30|New York|null|null|
+---+------------------+----+----+



In [30]:
splitcol=split(df["value"],",|\t|\|")
df.withColumn("id",splitcol.getItem(0)).\
   withColumn("Name",splitcol.getItem(1)).\
   withColumn("Age",splitcol.getItem(2)).\
   withColumn("City",splitcol.getItem(3)).select("id","Name","Age","City").show()

+---+-----+---+--------+
| id| Name|Age|    City|
+---+-----+---+--------+
|  1|Alice| 30|New York|
+---+-----+---+--------+



In [31]:
#Repeat above code with multiple delimeters with file load

In [60]:
#Missing values
data=[(1,),(2,),(4,),(5,),(7,),(8,),(10,)]
df=spark.createDataFrame(data,["id"])
df1=spark.range(1,11).toDF("id")
df1.join(df,"id","left_anti").show()
#Try with join and null
#df1.join(df,df.id==df.id,"left").select(df1.id.alias("full_id"),df.id).where(df.id.isnull()).show()

+---+
| id|
+---+
|  3|
|  6|
|  9|
+---+



In [61]:
#Top3 Movies
data_movies = [(1, "Movie A"), (2, "Movie B"), (3, "Movie C"), (4, "Movie D"), (5, "Movie E")]

data_ratings = [(1, 101, 4.5), (1, 102, 4.0), (2, 103, 5.0), 
                (2, 104, 3.5), (3, 105, 4.0), (3, 106, 4.0), 
                (4, 107, 3.0), (5, 108, 2.5), (5, 109, 3.0)]

columns_movies = ["MovieID", "MovieName"]
columns_ratings = ["MovieID", "UserID", "Rating"]

movies=spark.createDataFrame(data_movies,columns_movies)
ratings=spark.createDataFrame(data_ratings,columns_ratings)
movies.show()
ratings.show()

+-------+---------+
|MovieID|MovieName|
+-------+---------+
|      1|  Movie A|
|      2|  Movie B|
|      3|  Movie C|
|      4|  Movie D|
|      5|  Movie E|
+-------+---------+

+-------+------+------+
|MovieID|UserID|Rating|
+-------+------+------+
|      1|   101|   4.5|
|      1|   102|   4.0|
|      2|   103|   5.0|
|      2|   104|   3.5|
|      3|   105|   4.0|
|      3|   106|   4.0|
|      4|   107|   3.0|
|      5|   108|   2.5|
|      5|   109|   3.0|
+-------+------+------+



In [65]:
ratings.groupBy("MovieID").agg(avg(col("Rating")).alias("avg_rating")).orderBy("avg_rating",ascending=False).limit(3).show()

+-------+----------+
|MovieID|avg_rating|
+-------+----------+
|      1|      4.25|
|      2|      4.25|
|      3|       4.0|
+-------+----------+



In [67]:
#Moving Average
from pyspark.sql import Row
from pyspark.sql.window import Window

data = [Row(Date='2023-01-01', ProductID=100, QuantitySold=10),
        Row(Date='2023-01-02', ProductID=100, QuantitySold=15),
        Row(Date='2023-01-03', ProductID=100, QuantitySold=20),
        Row(Date='2023-01-04', ProductID=100, QuantitySold=25),
        Row(Date='2023-01-05', ProductID=100, QuantitySold=30),
        Row(Date='2023-01-06', ProductID=100, QuantitySold=35),
        Row(Date='2023-01-07', ProductID=100, QuantitySold=40),
        Row(Date='2023-01-08', ProductID=100, QuantitySold=45)]

# Create DataFrame
df_sales = spark.createDataFrame(data)
df_sales.show()

+----------+---------+------------+
|      Date|ProductID|QuantitySold|
+----------+---------+------------+
|2023-01-01|      100|          10|
|2023-01-02|      100|          15|
|2023-01-03|      100|          20|
|2023-01-04|      100|          25|
|2023-01-05|      100|          30|
|2023-01-06|      100|          35|
|2023-01-07|      100|          40|
|2023-01-08|      100|          45|
+----------+---------+------------+



In [70]:
# Convert Date string to Date type
df_sales = df_sales.withColumn("Date", to_date(col("Date")))

# Window specification for 7-day rolling average
windowSpec = Window.partitionBy('ProductID').orderBy('Date').rowsBetween(-6, 0)

# Calculating the rolling average
rollingAvg = df_sales.withColumn('7DayAvg', avg('QuantitySold').over(windowSpec))

# Show results
rollingAvg.show()

+----------+---------+------------+-------+
|      Date|ProductID|QuantitySold|7DayAvg|
+----------+---------+------------+-------+
|2023-01-01|      100|          10|   10.0|
|2023-01-02|      100|          15|   12.5|
|2023-01-03|      100|          20|   15.0|
|2023-01-04|      100|          25|   17.5|
|2023-01-05|      100|          30|   20.0|
|2023-01-06|      100|          35|   22.5|
|2023-01-07|      100|          40|   25.0|
|2023-01-08|      100|          45|   30.0|
+----------+---------+------------+-------+



In [18]:
data=[("vivek"),("supriya"),("akash")]
type(data)

list

In [17]:
data=[(1,"vivek"),(2,"supriya"),(3,"akash")]
type(data)

list

In [26]:
#Pyspark UDF
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

data = [Row(UserID=4001, Age=17),
        Row(UserID=4002, Age=45),
        Row(UserID=4003, Age=65),
        Row(UserID=4004, Age=30),
        Row(UserID=4005, Age=80)]

df=spark.createDataFrame(data)

def categorize(age):
    if age<18:
        return 'Youth'
    if age<60:
        return 'Adult'
    else:
        return 'Senior'
        
ageudf=udf(categorize,StringType())

df.withColumn("AgeGroup",ageudf(col("Age"))).show()
    

+------+---+--------+
|UserID|Age|AgeGroup|
+------+---+--------+
|  4001| 17|   Youth|
|  4002| 45|   Adult|
|  4003| 65|  Senior|
|  4004| 30|   Adult|
|  4005| 80|  Senior|
+------+---+--------+



In [28]:
#Find unique visitors to the website
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import countDistinct

# Sample data
visitor_data = [Row(Date='2023-01-01', VisitorID=101),
                Row(Date='2023-01-01', VisitorID=102),
                Row(Date='2023-01-01', VisitorID=101),
                Row(Date='2023-01-02', VisitorID=103),
                Row(Date='2023-01-02', VisitorID=101)]

# Create DataFrame
df_visitors = spark.createDataFrame(visitor_data)
df_visitors.show()

+----------+---------+
|      Date|VisitorID|
+----------+---------+
|2023-01-01|      101|
|2023-01-01|      102|
|2023-01-01|      101|
|2023-01-02|      103|
|2023-01-02|      101|
+----------+---------+



In [35]:
df_visitors.groupBy(col("Date")).agg(countDistinct(col("VisitorID")).alias("Unique_Visitors")).show()

+----------+---------------+
|      Date|Unique_Visitors|
+----------+---------------+
|2023-01-01|              2|
|2023-01-02|              2|
+----------+---------------+



In [36]:
# Determine the first purchase date for each user.

from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import min

# Sample data
purchase_data = [
    Row(UserID=1, PurchaseDate='2023-01-05'),
    Row(UserID=1, PurchaseDate='2023-01-10'),
    Row(UserID=2, PurchaseDate='2023-01-03'),
    Row(UserID=3, PurchaseDate='2023-01-12')
]

# Create DataFrame
df_purchases = spark.createDataFrame(purchase_data)
df_purchases.show()

+------+------------+
|UserID|PurchaseDate|
+------+------------+
|     1|  2023-01-05|
|     1|  2023-01-10|
|     2|  2023-01-03|
|     3|  2023-01-12|
+------+------------+



In [37]:
df_purchases.groupBy(col("UserID")).agg(min(col("PurchaseDate")).alias("FirstPurchaseDay")).show()

+------+----------------+
|UserID|FirstPurchaseDay|
+------+----------------+
|     1|      2023-01-05|
|     2|      2023-01-03|
|     3|      2023-01-12|
+------+----------------+



In [38]:
# Generate a sequential number for each row within each group, ordered by date.

from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

# Sample data
group_data = [
    Row(GroupID='A', Date='2023-01-01'),
    Row(GroupID='A', Date='2023-01-02'),
    Row(GroupID='B', Date='2023-01-01'),
    Row(GroupID='B', Date='2023-01-03')
]
df=spark.createDataFrame(group_data)
df.show()

+-------+----------+
|GroupID|      Date|
+-------+----------+
|      A|2023-01-01|
|      A|2023-01-02|
|      B|2023-01-01|
|      B|2023-01-03|
+-------+----------+



In [40]:
windowSpec=Window.partitionBy(col("GroupID")).orderBy(col("Date"))

df.withColumn("SeqNo",row_number().over(windowSpec)).show()

+-------+----------+-----+
|GroupID|      Date|SeqNo|
+-------+----------+-----+
|      A|2023-01-01|    1|
|      A|2023-01-02|    2|
|      B|2023-01-01|    1|
|      B|2023-01-03|    2|
+-------+----------+-----+



In [41]:
#Problem: Given a dataset of sales records, identify and replace all missing values in the 'amount' column with the average sales amount.

from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, col

# Initialize Spark Session
#spark = SparkSession.builder.appName("HandleMissingValues").getOrCreate()

# Sample data for sales - id, amount (with missing values represented by None)
sales_data = [("1", 100), ("2", 150), ("3", None), ("4", 200), ("5", None)]

# Creating DataFrame
sales_df = spark.createDataFrame(sales_data, ["sale_id", "amount"])

sales_df.show()

+-------+------+
|sale_id|amount|
+-------+------+
|      1|   100|
|      2|   150|
|      3|  null|
|      4|   200|
|      5|  null|
+-------+------+



In [53]:
#Solution1
avg_amt=sales_df.select(avg(col("amount"))).first()[0]
sales_df.fillna(avg_amt).show()

+-------+------+
|sale_id|amount|
+-------+------+
|      1|   100|
|      2|   150|
|      3|   150|
|      4|   200|
|      5|   150|
+-------+------+



In [52]:
#Solution2
avg_amount = sales_df.na.drop().agg(mean(col("amount"))).first()[0]

# Replace missing values with the average amount
sales_df_filled = sales_df.na.fill(avg_amount)

# Show the result
sales_df_filled.show()


+-------+------+
|sale_id|amount|
+-------+------+
|      1|   100|
|      2|   150|
|      3|   150|
|      4|   200|
|      5|   150|
+-------+------+



In [54]:
#Problem: Given a dataset of sales records with monthly sales per product, reshape the data to have one row per product-month combination.

from pyspark.sql.functions import expr

# Sample data: Product sales per month
data = [("Product1", 100, 150, 200),
        ("Product2", 200, 250, 300),
        ("Product3", 300, 350, 400)]

# Columns: Product, Sales_Jan, Sales_Feb, Sales_Mar
columns = ["Product", "Sales_Jan", "Sales_Feb", "Sales_Mar"]
df=spark.createDataFrame(data,columns)
df.show()

+--------+---------+---------+---------+
| Product|Sales_Jan|Sales_Feb|Sales_Mar|
+--------+---------+---------+---------+
|Product1|      100|      150|      200|
|Product2|      200|      250|      300|
|Product3|      300|      350|      400|
+--------+---------+---------+---------+



In [55]:
pivoted_df = df.selectExpr("Product", 
                           "stack(3, 'Jan', Sales_Jan, 'Feb', Sales_Feb, 'Mar', Sales_Mar) as (Month, Sales)")
pivoted_df.show()

+--------+-----+-----+
| Product|Month|Sales|
+--------+-----+-----+
|Product1|  Jan|  100|
|Product1|  Feb|  150|
|Product1|  Mar|  200|
|Product2|  Jan|  200|
|Product2|  Feb|  250|
|Product2|  Mar|  300|
|Product3|  Jan|  300|
|Product3|  Feb|  350|
|Product3|  Mar|  400|
+--------+-----+-----+



In [57]:
pivoted_df.groupBy("Product").pivot("Month").sum("Sales").show()

+--------+---+---+---+
| Product|Feb|Jan|Mar|
+--------+---+---+---+
|Product3|350|300|400|
|Product2|250|200|300|
|Product1|150|100|200|
+--------+---+---+---+

